<sup>This notebook is originally from https://github.com/lizanafj/UHI_mapping and licensed for reuse under [GPL-3.0 license].</sup>

# Data Analytics
---

This notebook has different data analysis techniques for the diagnosis of the urban heat island in cities

In [ ]:
import os
import requests
import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from matplotlib.dates import date2num

import datetime
from datetime import datetime

# get folder location of script
cwd = os.getcwd()

## Define name of city project to work
---

Define the name of the **`city`** (projectname) to upload/save all data during the workflow

In [ ]:
#City project: 
city= "London"

## Import input data of the project - config.py 
---

In [ ]:
os.chdir(cwd)

In [ ]:
cwd_main = cwd[:-10]
cwd_project = cwd_main + f"\projectname\{city}"
print(cwd_main)
print(cwd_project)

In [ ]:
#import config.py file

# go to current location of script
os.chdir(cwd_project)
import config as cf

In [ ]:
#check input variables
print("city: ",cf.city)
print("first date: ",cf.first_date)
print("last date: ",cf.last_date)
print("lat,long: ",cf.lat,cf.long)
print("plot: ",cf.plot)

## Update input data file (json file)
---

Update projectdata.json file with project variables

In [ ]:
os.chdir(cf.cwd_scripts_process)

In [ ]:
#update input variables (json file) in script folder for extraction
d = {'city':cf.city,'lat':cf.lat,'long':cf.long, 
     'plot':cf.plot, 
     "first_date":cf.first_date,"last_date":cf.last_date,
     "first_date1":cf.first_date1,"last_date1":cf.last_date1,
    "color_net":cf.color_net, "color_wund":cf.color_wund,
    "color_ows":cf.color_ows, "color_cws":cf.color_cws,
    "color_outliers":cf.color_outliers}

with open('projectdata_p.json', 'w') as fp:
    json.dump(d, fp)

## Data analytics
---

Run analytics

In [ ]:
os.chdir(cf.cwd_scripts_process)

%run C1_DataAnalytics_1.py

## Explore the results
---

Check files generated and outputs

In [ ]:
os.chdir(cf.cwd_data_proc)
os.listdir()

In [ ]:
#Read data
temp_CWS = pd.read_csv("Temperature_London_2021_CWS_all_QC_G8_proc.csv", index_col='date',parse_dates=True)
temp_rural = pd.read_csv("xTemperature_London_2021_CWS_rural_stations.csv", index_col='date',parse_dates=True)

In [ ]:
temp_rural

In [ ]:
os.chdir(cf.cwd_data_filt)
os.listdir()

In [ ]:
#Read data
temp_OWS = pd.read_csv("Temperature_London_2021_OWS_QC_I5.csv", index_col='date',parse_dates=True)

In [ ]:
temp_OWS

In [ ]:
#Figure to check day
day1="2021-08-01"
day2="2021-08-15"

fig, ax = plt.subplots(figsize=(18, 6))

# Add x-axis and y-axis
l1 = ax.plot(temp_rural.truncate(before=day1, after=day2).iloc[:,0],color='red',alpha=1,label="Mean temperature in surrounding rural area",linewidth=1.5)
l2 = ax.plot(temp_CWS.truncate(before=day1, after=day2).iloc[:,0],color='blue',alpha=1,label="CWS data",linewidth=1)
l3 = ax.plot(temp_OWS.truncate(before=day1, after=day2).iloc[:,0],color='k',alpha=1,label="OWS data",linewidth=1)

l4 = ax.plot(temp_CWS.truncate(before=day1, after=day2),color='blue',alpha=0.1,label='_Hidden',linewidth=0.2)
l5 = ax.plot(temp_OWS.truncate(before=day1, after=day2),color='k',alpha=1,label='_Hidden',linewidth=0.6)
l6 = ax.plot(temp_rural.truncate(before=day1, after=day2),color='red',alpha=1,label='_Hidden',linewidth=1.5)


# Set title and labels for axes
ax.set(ylabel="Temperature (ºC)",
       title="Overlapping of air temperature time series of CWS, OWS, and rural climate")

#ax.set(ylim=(5, 40))

plt.legend(loc=1)
#plt.grid(color='grey', linestyle='--', linewidth=0.6)

#margins - 0 applied
plt.margins(x=0)

# Define the date format
date_form = DateFormatter("%d-%m-%y")
ax.xaxis.set_major_formatter(date_form)

# Ensure a major tick for each week using (interval=1) 
ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))

os.chdir(cf.cwd_figures)
plt.savefig(f"03_Analytics_{city}_figure1.jpg", format='jpg')
plt.show()